# **Autoencoders y espacio latente**

Nota: este notebook está basado en este [código](https://blog.keras.io/building-autoencoders-in-keras.html)


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from keras.utils.vis_utils import plot_model
from tensorflow.keras.datasets import mnist
from IPython.display import Image
import imageio
import numpy as np


A continuación vamos a crear un autoencoder convolucional para el dataset **MNIST**

In [ ]:
input_img = Input(shape=(28, 28, 1))  

x = Conv2D(4, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
encoder = Model(input_img, encoded, name="encoder")

latent_code = Input(shape=(4, 4, 16))  
x = Conv2D(16, (3, 3), activation='relu', padding='same')(latent_code)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(4, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoder = Model(latent_code, decoded, name="decoder")

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
plot_model(autoencoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
encoder(x_test[None,1]).shape

In [ ]:
decoder(encoder(x_test[None,1])).shape

In [ ]:
idx1 = np.random.randint(x_test.shape[0])
idx2 = np.random.randint(x_test.shape[0])
x1 = x_test[idx1][None,:]
x2 = x_test[idx2][None,:]
plt.figure()
plt.subplot(1,2, 1)
plt.imshow(x1[0,:,:,0], cmap='gray')
plt.subplot(1,2, 2)
plt.imshow(x2[0,:,:,0], cmap='gray')

### Interpolar en el espacio de los pixeles

In [ ]:
#@title Default title text { run: "auto" }
alpha = 0.53 #@param {type:"slider", min:0, max:1, step:0.01}
pixel_diff = x1 + alpha*(x2-x1)
plt.imshow(pixel_diff[0,:,:,0], cmap='gray')

In [ ]:
gif = []
for alpha in np.linspace(0,1,100):
  pixel_diff = x1 + alpha*(x2-x1)
  gif.append(np.uint8(pixel_diff[0,:,:,0]*255))
try:
  imageio.mimsave('im.gif', gif)
except:
  pass

Image('im.gif', format='png', width=255)

### Interpolar en el espacio latente

In [ ]:
#@title Default title text { run: "auto" }
alpha = 0.25 #@param {type:"slider", min:0, max:1, step:0.01}
code1 = encoder(x1)
code2 = encoder(x2)
latent_diff = code1 + alpha*(code2-code1)
decoded_latent_diff = decoder(latent_diff)
decoded_latent_diff.shape
plt.imshow(decoded_latent_diff[0,:,:,0], cmap='gray')

In [ ]:
gif = []
code1 = encoder(x1)
code2 = encoder(x2)
for alpha in np.linspace(0,1,100):
  latent_diff = code1 + alpha*(code2-code1)
  decoded_latent_diff = decoder(latent_diff).numpy()
  gif.append(np.uint8(decoded_latent_diff[0,:,:,0]*255))
try:
  imageio.mimsave('im.gif', gif)
except:
  pass

Image('im.gif', format='png', width=255)

In [ ]:
#@title Default title text { run: "auto" }
alpha = 56.83 #@param {type:"slider", min:0, max:100, step:0.01}

code1 = encoder(x1).numpy()

code1[0, 1, 1, 10]=alpha

latent_diff = code1 
decoded_latent_diff = decoder(latent_diff)
decoded_latent_diff.shape
plt.imshow(decoded_latent_diff[0,:,:,0], cmap='gray')